https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [1]:
import sqlalchemy

In [3]:
sqlalchemy.__version__

'1.3.7'

In [4]:
from sqlalchemy import create_engine

In [5]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [6]:
engine

Engine(sqlite:///:memory:)

In [7]:
from sqlalchemy.ext.declarative import declarative_base

声明一些表，再声明一些类把它们关联起来是传统的做法，现代sqlalchemy通过引入declarative使得两者可以合二为一。简化操作。

In [8]:
Base = declarative_base()

In [9]:
from sqlalchemy import Column, Integer, String

In [10]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)
    


sqlalchemy不以类名和表名作任何匹配，而使用`__tablename__`进行显式映射，虽然这引入了一些boilerplate。但具有更大的好处，毕竟我们经常被给定数据库而不是在新建数据库，灵活性是重要的。

In [11]:
User

__main__.User

In [12]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [13]:
from pprint import pprint

In [14]:
pprint(User.__table__)

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)


定义好的Declarative会创建Table对象，并构建Mapper对象表示两者的映射。不过我们一般不需要和它们直接打交道，在对象API中。

Table对象属于MetaData对象，MetaData抽象一种元数据，既然有了元数据就已经可以用经典ORM的方式进行一些操作了，比如当前因为还没有表，可以用create_all把表创建到某个连接上（不错，定义本身和连接是分离的）。

In [15]:
Base.metadata.create_all(engine)

2019-08-15 14:27:58,448 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-15 14:27:58,449 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:27:58,450 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-08-15 14:27:58,451 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:27:58,452 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-08-15 14:27:58,452 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:27:58,454 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2019-08-15 14:27:58,454 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:27:58,456 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-08-15 14:27:58,457 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:27:58,458 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
ed_user = User(name='ed', fullname="Ed Jones", nickname="edsnickname")

In [17]:
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [18]:
ed_user.name

'ed'

In [19]:
ed_user.nickname

'edsnickname'

In [20]:
str(ed_user.id)

'None'

通过元类，User类已经获得了默认的__init__定义

In [21]:
from sqlalchemy.orm import sessionmaker

In [22]:
Session = sessionmaker(bind=engine)

可以看到Session以类似Base的方式被创建出来，可能是为了针对不同的backend定义不同的行为。

In [23]:
Session = sessionmaker()

In [24]:
Session.configure(bind=engine)

In [25]:
session = Session()

In [26]:
ed_user = User(name='ed', fullname="Ed Jones", nickname="edsnickname")

In [27]:
session.add(ed_user)

In [28]:
our_user = session.query(User).filter_by(name='ed').first()

2019-08-15 14:27:58,576 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-15 14:27:58,578 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-08-15 14:27:58,579 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-08-15 14:27:58,580 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-08-15 14:27:58,581 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [29]:
our_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [30]:
ed_user is our_user

True

看来这ORM还挺智能。。

延迟求值，汇总处理是缓解IO问题的经典策略。这里add也只是进入pending状态，直到flush或其他需要立即flush才能保证正确性的操作出现。

session这里起到某种恒等映射的作用，同一个访问的对象会先放到一个缓存池中，这样选出的同一个对象is同一个对象，这可以排除一些微妙的错误。

In [31]:
session.add_all([User(name = "wendy", fullname="Wendy Williams", nickname='windy'),
                User(name='mary', fullname='Mary Contrary', nickname='mary'),
                User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [32]:
ed_user.nickname = 'eddie'

In [33]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [34]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [35]:
session.commit()

2019-08-15 14:27:58,642 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2019-08-15 14:27:58,644 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2019-08-15 14:27:58,645 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-08-15 14:27:58,645 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2019-08-15 14:27:58,646 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-08-15 14:27:58,647 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2019-08-15 14:27:58,647 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-08-15 14:27:58,648 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2019-08-15 14:27:58,649 INFO sqlalchemy.engine.base.Engine COMMIT


这里我们看到session保持对象一致性的作用，只有这样我们才能使用修改一个对象属性来简介发起一个update的操作，否则如果多个对象进行不同的修改，看起来没有自然的处理方式（使用最后一个不如说是一种bug）。

In [36]:
ed_user.id

2019-08-15 14:27:58,655 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-15 14:27:58,656 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-08-15 14:27:58,657 INFO sqlalchemy.engine.base.Engine (1,)


1

In [37]:
ed_user.id

1

In [38]:
session.query(User).filter_by(name = 'mary').first().id

2019-08-15 14:27:58,675 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-08-15 14:27:58,676 INFO sqlalchemy.engine.base.Engine ('mary', 1, 0)


3

In [39]:
ed_user.name = "Edwardo"

In [40]:
fake_user = User(name='fakeuser', fullname="Invalid", nickname='12345')

In [41]:
session.add(fake_user)

In [42]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2019-08-15 14:27:58,716 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2019-08-15 14:27:58,717 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2019-08-15 14:27:58,719 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-08-15 14:27:58,719 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2019-08-15 14:27:58,720 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-08-15 14:27:58,721 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [43]:
session.rollback()

2019-08-15 14:27:58,729 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [44]:
ed_user.name

2019-08-15 14:27:58,738 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-15 14:27:58,740 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-08-15 14:27:58,741 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [45]:
fake_user in session

False

In [46]:
session.query(User).filter(User.name.in_(['ed','fakeuser'])).all()

2019-08-15 14:27:58,761 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2019-08-15 14:27:58,762 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

In [47]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2019-08-15 14:27:58,771 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2019-08-15 14:27:58,771 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


User的意思其实是搜索User的所有字段，我们也可以搜索部分。

In [48]:
for name,fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-08-15 14:27:58,783 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-08-15 14:27:58,783 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [49]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2019-08-15 14:27:58,794 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2019-08-15 14:27:58,795 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


In [50]:
from sqlalchemy.orm import aliased

In [51]:
user_alias = aliased(User, name='user_alias')

In [52]:
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2019-08-15 14:27:58,824 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.nickname AS user_alias_nickname 
FROM users AS user_alias
2019-08-15 14:27:58,827 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', nickname='eddie')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>


In [53]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2019-08-15 14:27:58,835 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2019-08-15 14:27:58,836 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [54]:
for name, in session.query(User.name).filter_by(fullname='Ed Jone'):
    print(name)

2019-08-15 14:27:58,846 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-08-15 14:27:58,848 INFO sqlalchemy.engine.base.Engine ('Ed Jone',)


In [55]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2019-08-15 14:27:58,856 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2019-08-15 14:27:58,857 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


In [56]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2019-08-15 14:27:58,865 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2019-08-15 14:27:58,866 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', nickname='eddie')>


In [57]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)

In [58]:
query

In [59]:
query.all()

2019-08-15 14:27:58,896 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-08-15 14:27:58,897 INFO sqlalchemy.engine.base.Engine ('%ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [60]:
query.first()

2019-08-15 14:27:58,905 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2019-08-15 14:27:58,907 INFO sqlalchemy.engine.base.Engine ('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='eddie')>

In [63]:
user = query.one()

2019-08-15 14:28:17,597 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2019-08-15 14:28:17,599 INFO sqlalchemy.engine.base.Engine ('%ed',)


MultipleResultsFound: Multiple rows were found for one()

In [64]:
query = session.query(User.id).filter(User.name == 'ed').order_by(User.id)

In [65]:
query.scalar()

2019-08-15 14:28:24,411 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = ? ORDER BY users.id
2019-08-15 14:28:24,412 INFO sqlalchemy.engine.base.Engine ('ed',)


1

In [66]:
from sqlalchemy import text

In [67]:
for user in session.query(User).filter(text("id<224")).order_by(text("id")).all():
    print(user.name)

2019-08-15 14:28:25,269 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE id<224 ORDER BY id
2019-08-15 14:28:25,270 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


固定大小的table能表示的对象很少，我们可能想要可选项和变长列表。在sql里这通过另一个表与之前的表的关联来实现。如我们设一个物品表，它的一个ForeignKey是User里的id，这其实表示了一种下属（树）关系，即User对物品是一个一对多的关系，而我们知道，只有一个父节点的图是树。

In [68]:
from sqlalchemy import ForeignKey

In [69]:
from sqlalchemy.orm import relationship

In [70]:
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    


In [71]:
User.addresses = relationship("Address", order_by = Address.id, back_populates="user")

relationship(target_table_name, ...)显然是指定本表与另一个表的关系，再给定另一个表中，对本表的称呼。这个可不可以与attr不一致看不出来。

In [72]:
Base.metadata.create_all(engine)

2019-08-15 14:28:27,542 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-08-15 14:28:27,543 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:28:27,544 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2019-08-15 14:28:27,545 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:28:27,546 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2019-08-15 14:28:27,546 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:28:27,547 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-08-15 14:28:27,548 INFO sqlalchemy.engine.base.Engine ()
2019-08-15 14:28:27,549 INFO sqlalchemy.engine.base.Engine COMMIT


In [73]:
jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')
jack

<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>

In [74]:
jack.addresses

[]

In [75]:
jack.addresses = [Address(email_address="jack@google.com"), Address(email_address='j25@yahoo.com')]

In [76]:
jack.addresses[1]

<Address(email_address='j25@yahoo.com')>

In [77]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>

In [78]:
session.add(jack)

In [79]:
session.commit()

2019-08-15 14:29:11,558 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-08-15 14:29:11,560 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2019-08-15 14:29:11,562 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-08-15 14:29:11,563 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 5)
2019-08-15 14:29:11,564 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-08-15 14:29:11,564 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 5)
2019-08-15 14:29:11,566 INFO sqlalchemy.engine.base.Engine COMMIT


In [80]:
jack = session.query(User).filter_by(name = 'jack').one()

2019-08-15 14:29:42,583 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-15 14:29:42,585 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-08-15 14:29:42,586 INFO sqlalchemy.engine.base.Engine ('jack',)


In [81]:
jack

<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>

In [82]:
jack.addresses

2019-08-15 14:29:52,432 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2019-08-15 14:29:52,433 INFO sqlalchemy.engine.base.Engine (5,)


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [84]:
for u,a in session.query(User, Address).\
            filter(User.id==Address.user_id).\
            filter(Address.email_address=='jack@google.com').\
            all():
    print(u)
    print(a)

2019-08-15 14:32:11,545 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = ?
2019-08-15 14:32:11,546 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)
<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>
<Address(email_address='jack@google.com')>


In [85]:
session.query(User).join(Address).\
        filter(Address.email_address == 'jack@google.com').\
        all()

2019-08-15 14:32:52,793 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE addresses.email_address = ?
2019-08-15 14:32:52,794 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)


[<User(name='jack', fullname='Jack Bean', nickname='gjffdd')>]

以上是Join操作，即虚构的张开一个大表，可以把多表查询转化为单表查询问题。

In [86]:
session.delete(jack)

In [87]:
session.query(User).filter_by(name='jack').count()

2019-08-15 14:35:15,151 INFO sqlalchemy.engine.base.Engine UPDATE addresses SET user_id=? WHERE addresses.id = ?
2019-08-15 14:35:15,152 INFO sqlalchemy.engine.base.Engine ((None, 1), (None, 2))
2019-08-15 14:35:15,154 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2019-08-15 14:35:15,155 INFO sqlalchemy.engine.base.Engine (5,)
2019-08-15 14:35:15,157 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2019-08-15 14:35:15,157 INFO sqlalchemy.engine.base.Engine ('jack',)


0

In [88]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', "j25@yahoo.com"])).count()

2019-08-15 14:36:09,487 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2019-08-15 14:36:09,488 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


2

In [89]:
session.close()

2019-08-15 14:36:58,971 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [90]:
Base = declarative_base()

In [91]:
>>> class User(Base):
...     __tablename__ = 'users'
...
...     id = Column(Integer, primary_key=True)
...     name = Column(String)
...     fullname = Column(String)
...     nickname = Column(String)
...
...     addresses = relationship("Address", back_populates='user',
...                     cascade="all, delete, delete-orphan")
...
...     def __repr__(self):
...        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
...                                self.name, self.fullname, self.nickname)


In [92]:
>>> class Address(Base):
...     __tablename__ = 'addresses'
...     id = Column(Integer, primary_key=True)
...     email_address = Column(String, nullable=False)
...     user_id = Column(Integer, ForeignKey('users.id'))
...     user = relationship("User", back_populates="addresses")
...
...     def __repr__(self):
...         return "<Address(email_address='%s')>" % self.email_address


In [94]:
# load Jack by primary key
>>> jack = session.query(User).get(5)


2019-08-15 14:38:31,876 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-08-15 14:38:31,877 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-08-15 14:38:31,878 INFO sqlalchemy.engine.base.Engine (5,)


In [95]:
>>> del jack.addresses[1]


2019-08-15 14:38:37,448 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id
2019-08-15 14:38:37,450 INFO sqlalchemy.engine.base.Engine (5,)


In [96]:
>>> session.query(Address).filter(
...     Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])
... ).count()


2019-08-15 14:38:42,427 INFO sqlalchemy.engine.base.Engine DELETE FROM addresses WHERE addresses.id = ?
2019-08-15 14:38:42,428 INFO sqlalchemy.engine.base.Engine (2,)
2019-08-15 14:38:42,430 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2019-08-15 14:38:42,431 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


1

看上去因为没有commit就close了，所以状态等价于回滚了。删除列表项的pythonic方式就是 `del x[idx]`，它这也使其可以被采纳了。

In [97]:
session.delete(jack)

In [98]:
session.query(User).filter_by(name='jack').count()

2019-08-15 14:40:15,578 INFO sqlalchemy.engine.base.Engine DELETE FROM addresses WHERE addresses.id = ?
2019-08-15 14:40:15,579 INFO sqlalchemy.engine.base.Engine (1,)
2019-08-15 14:40:15,580 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2019-08-15 14:40:15,581 INFO sqlalchemy.engine.base.Engine (5,)
2019-08-15 14:40:15,582 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2019-08-15 14:40:15,583 INFO sqlalchemy.engine.base.Engine ('jack',)


0

In [99]:
>>> session.query(Address).filter(
...    Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])
... ).count()


2019-08-15 14:40:22,407 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2019-08-15 14:40:22,407 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


0

可以看到不加其他声明，delete删除并不会自动删除其“下属”的其他表中的项，这可以通过以上的`cascade="all, delete, delete-orphan"`声明解决，当然这个定义怎么看都很丑。

另外我们看到因为与我们顶层交互的始终是session（其创建出来时已经惰性求值的与某个数据库连接联系起来了）而不是某个大表，所以commit前，我们应该把某个节点放到session上或者移除，通过add或delete方法。或者query出的节点再commit，这样就可以完成基本的增删改。